<a href="https://colab.research.google.com/github/harryguiacorn/colab/blob/main/Ichimoku_chart_display.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance pandas plotly

# Download raw data and load into a data frame

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the ticker symbol (SPY) and end date
ticker_symbol = "SPY"
end_date = datetime.today().date()
start_date = datetime(end_date.year - 5, 1, 1)  # Beginning of the year 5 years ago

# Download historical data using yfinance
data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Format the dates as strings
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

# Save the data to a CSV file with dates in the filename
csv_filename = f"{ticker_symbol}_{start_date_str}-{end_date_str}_daily_data"
csv_filename_full = f"{csv_filename}.csv"

print(f"Data for {ticker_symbol} saved to {csv_filename_full}")
data.to_csv(csv_filename_full)

[*********************100%%**********************]  1 of 1 completed
Data for SPY saved to SPY_2018-01-01-2023-10-31_daily_data.csv


In [3]:

# Load the CSV data
data = pd.read_csv(csv_filename_full)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)


# Use Tapy to generate Ichimoku data

In [4]:
!pip install tapy
import pandas as pd
from tapy import Indicators
from datetime import datetime

# Apply Ichimoku analysis
# ichimoku = ta.Ichimoku()
# data = ichimoku(data)

__df_ichimoku = Indicators(data)
__df_ichimoku.ichimoku_kinko_hyo()  # column_name_kijun_sen="K Line"

# Save the data with Ichimoku analysis to a new CSV file
output_csv_filename = f"{csv_filename}_ichimoku_tapy"
output_csv_filename_full = f"{output_csv_filename}.csv"
data_ichimoku_tapy = __df_ichimoku.df
data_ichimoku_tapy.to_csv(output_csv_filename_full)
# print(data_ichimoku_tapy.head)
print(f"Ichimoku analysis for SPY data saved to {output_csv_filename_full}")


Ichimoku analysis for SPY data saved to SPY_2018-01-01-2023-10-31_daily_data_ichimoku_tapy.csv


# Use Finta to generate Ichimoku data

In [ ]:
!pip install finta

from finta import TA

# Finta works but doesn't produce future senkou a or b.
# df = pd.read_csv(csv_filename_full)
output_csv_filename = f"{csv_filename}_ichimoku_finta"
output_csv_filename_full = f"{output_csv_filename}.csv"
data_ichimoku_finta = TA.ICHIMOKU(data).to_csv(output_csv_filename_full)

# Plot on a chart

## Line Chart

In [6]:
import plotly.express as px

# Create an interactive chart
fig = px.line(data_ichimoku_tapy, x=data.index, y='Close', title='SPY Daily Closing Prices')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Close Price')
fig.show()

## Interactive Chart

In [5]:
import plotly.graph_objects as go

# Calculate Ichimoku cloud data
conversion_period = 9
base_period = 26
lagging_span = 26
displacement = 26

data['tenkan_sen'] = data_ichimoku_tapy['tenkan_sen']
data['kijun_sen'] = data_ichimoku_tapy['kijun_sen']
data['senkou_span_a'] = data_ichimoku_tapy['senkou_span_a']
data['senkou_span_b'] = data_ichimoku_tapy['senkou_span_b']

# Create the interactive Ichimoku cloud chart
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'],
                name='Candlesticks'))

# Add the Tenkan-sen and Kijun-sen lines
# fig.add_trace(go.Scatter(x=data.index, y=data['tenkan_sen'], mode='lines', name='Tenkan-sen', line=dict(color='blue')))
# fig.add_trace(go.Scatter(x=data.index, y=data['kijun_sen'], mode='lines', name='Kijun-sen', line=dict(color='red')))

# Add Senkou Span A and Senkou Span B as filled areas
fig.add_trace(go.Scatter(x=data.index, y=data['senkou_span_a'], fillcolor='rgba(0,255,0,0.2)', line=dict(width=2), name='Senkou Span A'))
fig.add_trace(go.Scatter(x=data.index, y=data['senkou_span_b'], fill='tonexty', fillcolor='rgba(255,0,0,0.2)', line=dict(width=2), name='Senkou Span B')) # fill area between trace0 and trace1

# Update layout
fig.update_layout(
    title=f'{ticker_symbol} Ichimoku Cloud Chart',
    xaxis_rangeslider_visible=False
)

# Show the interactive chart
fig.show()
